In [ ]:
import csv
import json
import re
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 設定 Chrome options
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 不開啟瀏覽器 (可選)
options.add_argument("--disable-gpu")
options.add_argument("--log-level=3")  # 降低日誌級別
options.add_argument("--enable-logging")
options.add_argument("--v=1")

# 啟用 Performance Logging
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

# 啟動 WebDriver
driver = webdriver.Chrome(options=options)

# 進入 Google Maps 景點評論頁面
url = "https://www.google.com.tw/maps/place/%E5%9C%8B%E7%AB%8B%E6%95%85%E5%AE%AE%E5%8D%9A%E7%89%A9%E9%99%A2/@25.1023602,121.5459122,17z/data=!3m1!5s0x3442ac3b2ddb9a43:0x7a84c798191dd2cf!4m18!1m9!3m8!1s0x3442ac3acd404a7d:0x5d6d7018397a09c1!2z5ZyL56uL5pWF5a6u5Y2a54mp6Zmi!8m2!3d25.1023554!4d121.5484925!9m1!1b1!16zL20vMGhod2w!3m7!1s0x3442ac3acd404a7d:0x5d6d7018397a09c1!8m2!3d25.1023554!4d121.5484925!9m1!1b1!16zL20vMGhod2w?hl=zh-TW&entry=ttu&g_ep=EgoyMDI1MDExNS4wIKXMDSoASAFQAw%3D%3D"
driver.get(url)
time.sleep(5)

# 儲存擷取到的URL，避免重複
collected_urls = set()

# 點擊排序按鈕，設置為"最新"排序
try:
    sort_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-value='排序']"))
    )
    sort_button.click()
    time.sleep(1)

    latest_button = driver.find_elements(By.CSS_SELECTOR, "div.mLuXec")[1]
    driver.execute_script("arguments[0].scrollIntoView();", latest_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", latest_button)
    time.sleep(2)
except Exception as e:
    print("無法更改排序為最新:", e)
    driver.quit()

# 模擬滾動並擷取請求 URL
for i in range(100):
    try:
        comments_container = driver.find_element(By.CSS_SELECTOR, "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde")

        # 隨機往下滑動
        scroll_down = random.randint(700, 1000)
        driver.execute_script(f"arguments[0].scrollBy(0, {scroll_down});", comments_container)
        time.sleep(random.uniform(0.5,1.5))  # 減少等待時間

        # 隨機往上滑動
        scroll_up = random.randint(100, 200)
        driver.execute_script(f"arguments[0].scrollBy(0, -{scroll_up});", comments_container)
        time.sleep(random.uniform(0.5,2))  # 減少等待時間

    except Exception as e:
        print("評論區容器滾動失敗:", e)

    # 擷取 Performance Log 並篩選出 'listugcposts' API
    logs = driver.get_log('performance')
    for entry in logs:
        try:
            log_message = json.loads(entry["message"])  # 解析日誌
            message_text = log_message.get("message", {}).get("params", {}).get("request", {}).get("url", "")
            # 使用正則表達式擷取所需的 URL
            match = re.search(r'https://www\.google\.com\.tw/maps/rpc/listugcposts\?[^"]+', message_text)
            if match:
                url = match.group(0)
                collected_urls.add(url)
        except (json.JSONDecodeError, KeyError):
            continue

driver.quit()

# 將擷取到的URL輸出至 console 並存入 CSV 檔案
output_file = "google_maps_review_urls.csv"

# 寫入 CSV 檔
with open(output_file, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["URL"])  # 寫入標題
    for url in collected_urls:
        print(url)  # 在 console 顯示 URL
        writer.writerow([url])

print(f"已擷取 {len(collected_urls)} 個評論請求 URL，並存入 {output_file}")
